# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.experiment import Experiment
import os

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-ml'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
key = "forestfire"
dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
y = df["Prediction"]
X = df.drop("Prediction", axis =1)

## Create or Attach an AmlCompute cluster

In [ ]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "hyperdrive-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1, delay_evaluation=5 )

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "C": choice(0.1, 1, 5, 10),
    "max_iter": choice(500, 1000, 4000, 10000)
}
)

#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=compute_target, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "hyperdrive1")
run = experiment.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

In [ ]:
src_bestrun = ScriptRunConfig(source_directory=".", script="train.py", compute_target=cpu_cluster, environment=sklearn_env,arguments=['--C',0.1, '--max_iter',1000])
experiment = Experiment(ws, "bestrun1")
run2 = experiment.submit(config=src_bestrun)
run2.wait_for_completion(show_output=True)

In [ ]:
#TODO: Save the best model
best_run.download_file(name="hyperdrivemodel.pkl")

In [ ]:
model = best_run.register_model(model_name="hd_model")
print(model.name, model.id, model.version, sep='\t')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
print(run2.get_file_names())

# register model 
model = run2.register_model(model_name='hyperdrive_model', model_path='outputs/hyperdrivemodel.pkl')
print(model.name, model.id, model.version, sep='\t')

# create environment for the deploy
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice

# get a curated environment
env = Environment.get(
    workspace=ws, 
    name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference",
    version=1
)
env.inferencing_stack_version='latest'

# create deployment config i.e. compute resources
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description="hyperdrive_for_email",
)


%%time
import uuid
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import Model

# get the registered model
model = Model(ws, "outputs/hyperdrivemodel.pkl")

# create an inference config i.e. the scoring script and environment
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# deploy the service
service_name = "hyperdrive_email"
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# send raw HTTP request to test the web service.
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X) - 1)
input_data = '{"data": [' + str(list(X[random_index])) + "]}"

headers = {"Content-Type": "application/json"}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("label:", y[random_index])
print("prediction:", resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
resp.raise_for_status()

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

